In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range

from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [2]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [3]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

9.999999999998899e-05

In [4]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

9e-05

In [5]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsFeature(df)
    df = ATRFeature(df)
    df = KeltnerChannelsFeature(df)
    df = RSIFeature(df)
    df = MACDFeature(df)
    df['pivots_l'] = False
    df['pivots_h'] = False

    df['low_perv'] = df.low.shift(1)
    df['high_perv'] = df.high.shift(1)
    df['low_next'] = df.low.shift(-1)
    df['high_next'] = df.high.shift(-1)
    df['pivots_l'] = (df.low_perv > df.low) & (df.low_next > df.low)
    df['pivots_h'] = (df.high_perv < df.high) & (df.high_next < df.high)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    df.pivots_l = df.pivots_l.astype(float)
    df.pivots_h = df.pivots_h.astype(float)
    return df

In [6]:
df_an = featureExtraction(df_an)

In [7]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = BUY
                    break
        
        df_an['SL'][i] = df_an.low[i-1] 
        df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]
                    
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = SELL
                    break
                    
        df_an['SL'][i] = df_an.high[i-1] 
        df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    

C:\Users\meysam\AppData\Local\Temp\ipykernel_8584\3582309630.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['SL'][i] = df_an.high[i-1]
C:\Users\meysam\AppData\Local\Temp\ipykernel_8584\3582309630.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
C:\Users\meysam\AppData\Local\Temp\ipykernel_8584\3582309630.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [8]:

feature_col = [
#     'open', 'high', 'low', 'close',
    
    'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c',
    'Feat_EMA_c', 'Feat_KeUp_c', 'Feat_KeLo_c',
    
    'Feat_BB_MA_o', 'Feat_BB_UP_o', 'Feat_BB_LW_o',
    'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 
    
    'Feat_BB_MA_l', 'Feat_BB_UP_l', 'Feat_BB_LW_l',
    'Feat_EMA_l', 'Feat_KeUp_l', 'Feat_KeLo_l',
    
    'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
    'Feat_EMA_h', 'Feat_KeUp_h', 'Feat_KeLo_h',
    
    'Feat_ATR_14',
    'Feat_gains','Feat_wins_rma', 'Feat_losses_rma', 'Feat_RSI_14', 
#     'Feat_MACD', 'Feat_SIGNAL_MACD', 'Feat_HIST',
    
        'full_range',# this is added ...
       'body_lower', 'body_upper', 
        'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
    'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [9]:
X = []
Y = [] 
indeces = []
for i in range(4,len(df_an)):
    if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        feature.append(df_an.pivots_h[i-2])
        feature.append(df_an.pivots_l[i-2])
        feature.append(df_an.pivots_h[i-3])
        feature.append(df_an.pivots_l[i-3])
        feature.append(df_an.pivots_h[i-4])
        feature.append(df_an.pivots_l[i-4])
        for fc in feature_col:
            feature.append(df_an[fc][i])
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [10]:
SPLIT_INDEX = int(0.8*len(X))
X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]


In [11]:
clf = MLPClassifier( solver='adam', alpha=1e-4,learning_rate_init=1e-4, # adam
                    hidden_layer_sizes=(200,200), 
                    random_state=1,
                    max_iter=2000,
                    verbose=True,warm_start = True,
                    n_iter_no_change=40,
#                     early_stopping = True,
#                     validation_fraction = 0.2 # default is 0.1
                   )

In [12]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 0.95059229
Iteration 2, loss = 0.88998637
Iteration 3, loss = 0.84624379
Iteration 4, loss = 0.81180068
Iteration 5, loss = 0.78524444
Iteration 6, loss = 0.76217921
Iteration 7, loss = 0.74459829
Iteration 8, loss = 0.73117578
Iteration 9, loss = 0.72271900
Iteration 10, loss = 0.71388224
Iteration 11, loss = 0.70928846
Iteration 12, loss = 0.70489855
Iteration 13, loss = 0.69920256
Iteration 14, loss = 0.69700332
Iteration 15, loss = 0.69278900
Iteration 16, loss = 0.69090581
Iteration 17, loss = 0.68850739
Iteration 18, loss = 0.68785291
Iteration 19, loss = 0.68587450
Iteration 20, loss = 0.68296422
Iteration 21, loss = 0.68057472
Iteration 22, loss = 0.67792771
Iteration 23, loss = 0.67631415
Iteration 24, loss = 0.67374663
Iteration 25, loss = 0.67249386
Iteration 26, loss = 0.67230569
Iteration 27, loss = 0.66953515
Iteration 28, loss = 0.66768726
Iteration 29, loss = 0.66663266
Iteration 30, loss = 0.66525341
Iteration 31, loss = 0.66299818
Iteration 32, los

Iteration 253, loss = 0.43229276
Iteration 254, loss = 0.42756442
Iteration 255, loss = 0.42843831
Iteration 256, loss = 0.42630120
Iteration 257, loss = 0.42940466
Iteration 258, loss = 0.42578101
Iteration 259, loss = 0.42308837
Iteration 260, loss = 0.42648707
Iteration 261, loss = 0.42596828
Iteration 262, loss = 0.42492410
Iteration 263, loss = 0.42023829
Iteration 264, loss = 0.42069900
Iteration 265, loss = 0.42006018
Iteration 266, loss = 0.41720606
Iteration 267, loss = 0.41696323
Iteration 268, loss = 0.41725309
Iteration 269, loss = 0.41718002
Iteration 270, loss = 0.41697436
Iteration 271, loss = 0.41340340
Iteration 272, loss = 0.41323240
Iteration 273, loss = 0.41552015
Iteration 274, loss = 0.41074807
Iteration 275, loss = 0.41126215
Iteration 276, loss = 0.41121536
Iteration 277, loss = 0.40827119
Iteration 278, loss = 0.41018062
Iteration 279, loss = 0.40961272
Iteration 280, loss = 0.40707797
Iteration 281, loss = 0.40664566
Iteration 282, loss = 0.40655686
Iteration 

Iteration 502, loss = 0.25517655
Iteration 503, loss = 0.25247134
Iteration 504, loss = 0.25199745
Iteration 505, loss = 0.25283804
Iteration 506, loss = 0.25289697
Iteration 507, loss = 0.25141753
Iteration 508, loss = 0.24957743
Iteration 509, loss = 0.25037334
Iteration 510, loss = 0.25020568
Iteration 511, loss = 0.24679815
Iteration 512, loss = 0.24863456
Iteration 513, loss = 0.24889551
Iteration 514, loss = 0.24884243
Iteration 515, loss = 0.24671619
Iteration 516, loss = 0.24598388
Iteration 517, loss = 0.24431845
Iteration 518, loss = 0.24271624
Iteration 519, loss = 0.24332111
Iteration 520, loss = 0.24256799
Iteration 521, loss = 0.24413056
Iteration 522, loss = 0.24340135
Iteration 523, loss = 0.24106591
Iteration 524, loss = 0.24726256
Iteration 525, loss = 0.24293189
Iteration 526, loss = 0.24047360
Iteration 527, loss = 0.24374089
Iteration 528, loss = 0.23764377
Iteration 529, loss = 0.24043824
Iteration 530, loss = 0.24183810
Iteration 531, loss = 0.23999016
Iteration 

Iteration 751, loss = 0.14079754
Iteration 752, loss = 0.14032256
Iteration 753, loss = 0.13883729
Iteration 754, loss = 0.13942432
Iteration 755, loss = 0.14187750
Iteration 756, loss = 0.14381075
Iteration 757, loss = 0.13944714
Iteration 758, loss = 0.13810940
Iteration 759, loss = 0.13952078
Iteration 760, loss = 0.13848014
Iteration 761, loss = 0.13888591
Iteration 762, loss = 0.13764821
Iteration 763, loss = 0.13533753
Iteration 764, loss = 0.13868511
Iteration 765, loss = 0.13823777
Iteration 766, loss = 0.13538958
Iteration 767, loss = 0.13350954
Iteration 768, loss = 0.13549015
Iteration 769, loss = 0.13258057
Iteration 770, loss = 0.13403514
Iteration 771, loss = 0.13147538
Iteration 772, loss = 0.13775118
Iteration 773, loss = 0.13425429
Iteration 774, loss = 0.13777280
Iteration 775, loss = 0.13713878
Iteration 776, loss = 0.13258952
Iteration 777, loss = 0.13377650
Iteration 778, loss = 0.13103380
Iteration 779, loss = 0.12877832
Iteration 780, loss = 0.12743778
Iteration 

Iteration 1000, loss = 0.06671019
Iteration 1001, loss = 0.06770560
Iteration 1002, loss = 0.06704662
Iteration 1003, loss = 0.06653155
Iteration 1004, loss = 0.06889190
Iteration 1005, loss = 0.06643258
Iteration 1006, loss = 0.06641979
Iteration 1007, loss = 0.06613126
Iteration 1008, loss = 0.06615759
Iteration 1009, loss = 0.06633816
Iteration 1010, loss = 0.06666515
Iteration 1011, loss = 0.06456407
Iteration 1012, loss = 0.06519064
Iteration 1013, loss = 0.06427201
Iteration 1014, loss = 0.06605009
Iteration 1015, loss = 0.06448053
Iteration 1016, loss = 0.06440878
Iteration 1017, loss = 0.06457584
Iteration 1018, loss = 0.06346698
Iteration 1019, loss = 0.06503696
Iteration 1020, loss = 0.06365141
Iteration 1021, loss = 0.06517092
Iteration 1022, loss = 0.06200813
Iteration 1023, loss = 0.06289725
Iteration 1024, loss = 0.06262919
Iteration 1025, loss = 0.06100050
Iteration 1026, loss = 0.06171122
Iteration 1027, loss = 0.06385920
Iteration 1028, loss = 0.06237890
Iteration 1029

Iteration 1241, loss = 0.02921715
Iteration 1242, loss = 0.02845349
Iteration 1243, loss = 0.02905618
Iteration 1244, loss = 0.02943566
Iteration 1245, loss = 0.02802853
Iteration 1246, loss = 0.02751952
Iteration 1247, loss = 0.02794128
Iteration 1248, loss = 0.02991151
Iteration 1249, loss = 0.02749183
Iteration 1250, loss = 0.02700046
Iteration 1251, loss = 0.02710629
Iteration 1252, loss = 0.02629996
Iteration 1253, loss = 0.02739656
Iteration 1254, loss = 0.02687819
Iteration 1255, loss = 0.02766028
Iteration 1256, loss = 0.02734299
Iteration 1257, loss = 0.02703398
Iteration 1258, loss = 0.02691762
Iteration 1259, loss = 0.02746089
Iteration 1260, loss = 0.02773186
Iteration 1261, loss = 0.02553256
Iteration 1262, loss = 0.02618664
Iteration 1263, loss = 0.02696769
Iteration 1264, loss = 0.02602700
Iteration 1265, loss = 0.02727868
Iteration 1266, loss = 0.02574722
Iteration 1267, loss = 0.02604681
Iteration 1268, loss = 0.02889024
Iteration 1269, loss = 0.02778394
Iteration 1270

Iteration 1482, loss = 0.01042056
Iteration 1483, loss = 0.01084947
Iteration 1484, loss = 0.01100022
Iteration 1485, loss = 0.01067759
Iteration 1486, loss = 0.01223192
Iteration 1487, loss = 0.01071331
Iteration 1488, loss = 0.01019778
Iteration 1489, loss = 0.01019853
Iteration 1490, loss = 0.01058824
Iteration 1491, loss = 0.01084326
Iteration 1492, loss = 0.01129981
Iteration 1493, loss = 0.01395306
Iteration 1494, loss = 0.01050607
Iteration 1495, loss = 0.00971561
Iteration 1496, loss = 0.00987237
Iteration 1497, loss = 0.00970861
Iteration 1498, loss = 0.01009847
Iteration 1499, loss = 0.00954183
Iteration 1500, loss = 0.00950896
Iteration 1501, loss = 0.00940409
Iteration 1502, loss = 0.00928312
Iteration 1503, loss = 0.00937664
Iteration 1504, loss = 0.00969416
Iteration 1505, loss = 0.00970746
Iteration 1506, loss = 0.00958551
Iteration 1507, loss = 0.00946793
Iteration 1508, loss = 0.01011667
Iteration 1509, loss = 0.00921961
Iteration 1510, loss = 0.00923615
Iteration 1511

Iteration 1723, loss = 0.00375588
Iteration 1724, loss = 0.00382527
Iteration 1725, loss = 0.00397038
Iteration 1726, loss = 0.00424944
Iteration 1727, loss = 0.00383898
Iteration 1728, loss = 0.00384678
Iteration 1729, loss = 0.00428288
Iteration 1730, loss = 0.00405770
Iteration 1731, loss = 0.00386814
Iteration 1732, loss = 0.00380508
Iteration 1733, loss = 0.00403257
Iteration 1734, loss = 0.00400527
Iteration 1735, loss = 0.00405358
Iteration 1736, loss = 0.00371342
Iteration 1737, loss = 0.00377286
Iteration 1738, loss = 0.00392437
Iteration 1739, loss = 0.00377725
Iteration 1740, loss = 0.00371332
Iteration 1741, loss = 0.00385210
Iteration 1742, loss = 0.00363504
Iteration 1743, loss = 0.00367836
Iteration 1744, loss = 0.00378431
Iteration 1745, loss = 0.00373049
Iteration 1746, loss = 0.00366742
Iteration 1747, loss = 0.00355433
Iteration 1748, loss = 0.00350080
Iteration 1749, loss = 0.00363417
Iteration 1750, loss = 0.00364578
Iteration 1751, loss = 0.00361473
Iteration 1752

MLPClassifier(hidden_layer_sizes=(200, 200), learning_rate_init=0.0001,
              max_iter=2000, n_iter_no_change=40, random_state=1, verbose=True,
              warm_start=True)

In [13]:
# np.savetxt("coefs_0.csv", clf.coefs_[0], delimiter=",")
# np.savetxt("coefs_1.csv", clf.coefs_[1], delimiter=",")
# np.savetxt("coefs_2.csv", clf.coefs_[2], delimiter=",")
# np.savetxt("intercepts_0.csv", clf.intercepts_[0], delimiter=",")
# np.savetxt("intercepts_1.csv", clf.intercepts_[1], delimiter=",")
# np.savetxt("intercepts_2.csv", clf.intercepts_[2], delimiter=",")

In [14]:
y_perdict = clf.predict(X_test)

In [15]:
clf.score(X_train, Y_train)

1.0

In [16]:
clf.score(X_test, Y_test)

0.553757225433526

In [17]:
conf = confusion_matrix(Y_test, y_perdict)
conf

array([[135, 207,   7],
       [159, 651, 236],
       [  9, 154, 172]], dtype=int64)

In [18]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
        if df_test.pivots_h[idx-1] or df_test.pivots_l[idx-1] :
            df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)

C:\Users\meysam\AppData\Local\Temp\ipykernel_8584\1801883316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.SIGNAL.loc[idx] = y_perdict[i]


In [19]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [20]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

prepare_data...
run_test...
Result:
win rate:  0.37716763005780346
-1.0    431
 2.0    261
Name: result, dtype: int64


In [23]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_8584\413402749.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_8584\413402749.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])

min balance  61.59953739124289
